# 1. 환경설정

In [1]:
import pickle
import time
import pandas as pd
import pickle
from pykiwoom.kiwoom import *
import warnings
warnings.filterwarnings('ignore')
kiwoom = Kiwoom()
kiwoom.CommConnect(block=True)

ModuleNotFoundError: No module named 'pykiwoom'

In [ ]:
state = kiwoom.GetConnectState()
if state == 0:
    print("미연결")
elif state == 1:
    print("연결완료")

# 2. 데이터 읽기

In [ ]:
with open('sector.pickle','rb') as f:
    sector = pickle.load(f)
with open('df_sector2.pickle','rb') as f:
    df_sector = pickle.load(f)
#with open('kospi.pickle','rb') as f:
 #   kospi = pickle.load(f)

In [ ]:
sector

In [ ]:
df_sector

In [ ]:
#kospi

# 3. 데이터셋 최신화
1. 최신 주가 데이터 읽고 업종 데이터셋 추가
2. 최신 코스피 지수 읽고 코스피 데이터셋 추가

In [ ]:
from datetime import date,datetime
today = date.today()
today = today.isoformat()
today = today.replace('-','')
#yesterday = int(today)-1
#yesterday = str(yesterday)
#print(yesterday, type(yesterday))
set_d = today

In [ ]:
today

In [ ]:
set_d

# 최신 data 수집

In [ ]:
df_sector_2 = {'Food':{},
          'Clothing' : {},
          'Chemical' : {},
          'Medicine' : {},
          'Non_Metal' : {},
          'Metal': {},
          'Machine' : {},
          'Electronic' : {},
          'Construction' : {},
          'Transport': {},
          'Distribution' : {},
          'Power': {},
          'Tele' : {},
          'Finance': {},
          'Brokerage' : {},
          'Insurer': {},
          'Service': {},
          'Manufacturer': {}}

In [ ]:
def Add_Data(sector_name,stockcode):
    df = kiwoom.block_request("opt10081",
                          종목코드=stockcode,
                          기준일자=set_d,
                          수정주가구분=1,
                          output="주식일봉차트조회",
                          next=0)
    df = df[['일자','현재가','거래량','시가','고가','저가']]
    df2 = pd.DataFrame()
    df2['일자'] = pd.to_datetime(df['일자'])
    df2['종가'] = df['현재가'].astype(int)
    df2['거래량'] = df['거래량'].astype(int)
    df2['시가'] = df['시가'].astype(int)
    df2['고가'] = df['고가'].astype(int)
    df2['저가'] = df['저가'].astype(int)
    df_sector_2[sector_name][stockcode] = df2
    del df2

In [ ]:
import time

In [ ]:
#데이터 수집 자동화 코드
for sector_name, stockcode_list in sector.items():
    print(f'---------{sector_name}업종 데이터 수집----------')
    for stockcode in stockcode_list:
        # 주식항목이름
        stockname = kiwoom.GetMasterCodeName(stockcode)
        print(f'--------[{stockname}] 항목 데이터 수집---------')
        Add_Data(sector_name,stockcode)
        time.sleep(0.2) #time을 안넣어주면 중간에 종종 수집이 안되고 멈추는 경우가 발생

In [ ]:
df_sector_2

# 이전 data와 최신 data 병합(최신화)

In [ ]:
'''
df_sector_3 = {'Food':{},
          'Clothing' : {},
          'Chemical' : {},
          'Medicine' : {},
          'Non_Metal' : {},
          'Metal': {},
          'Machine' : {},
          'Electronic' : {},
          'Construction' : {},
          'Transport': {},
          'Distribution' : {},
          'Power': {},
          'Tele' : {},
          'Finance': {},
          'Brokerage' : {},
          'Insurer': {},
          'Service': {},
          'Manufacturer': {}}
'''

In [ ]:
for sector_name, df_dict in df_sector.items():
    for stockcode, df in df_dict.items():
        df = pd.concat([df_sector_2[sector_name][stockcode],df])
        df=df.drop_duplicates(subset=['일자'])
        df = df.reset_index(drop=True)
        df_sector[sector_name][stockcode] = df

In [ ]:
df_sector

# 최신 kospi 수집

In [ ]:
'''
def Add_Kospi():
    df = kiwoom.block_request("opt20006",
                              업종코드="001",
                              기준일자=set_d,
                              output="업종일봉조회",
                              next=0)
    df = df[['일자','현재가']]
    df2 = pd.DataFrame()
    df2['일자'] = pd.to_datetime(df['일자'])
    df2['코스피지수'] = df['현재가'].astype(int)
    df2['코스피지수'] = df2['코스피지수'].apply(lambda x : x/100)
    return df2    
    
'''

In [ ]:
#kospi2 = Add_Kospi()

In [ ]:
#kospi2

# 이전 kospi와 최신 kospi 병합(최신화)

In [ ]:
#kospi3 = pd.concat([kospi2,kospi])
#kospi3=kospi3.drop_duplicates(subset=['일자'])
#kospi3 = kospi3.reset_index(drop=True)

In [ ]:
#kospi3

In [ ]:
with open('df_sector_UTD.pickle','wb') as f:
    pickle.dump(df_sector,f)

In [ ]:
# with open('kospi.pickle','wb') as f:
#     pickle.dump(kospi3,f)